In [ ]:
from pathlib import Path
import os
import re
import datetime
import warnings
from shutil import copyfile
from dataclasses import dataclass
import json
import pandas as pd
import numpy as np

BACKUP_DIR = Path("./backups")
UPOWER_BACKUP_DIR = BACKUP_DIR / "upower"

In [ ]:
json_file_names = os.listdir(UPOWER_BACKUP_DIR)
data = []
for filename in json_file_names:
    with open(UPOWER_BACKUP_DIR / filename, "r", encoding="utf8") as f:
        data.append(json.load(f))
len(json_file_names)
df = pd.DataFrame(data).sort_values("created_utc")
df['date'] = df['created_utc'].transform(datetime.datetime.fromtimestamp)

In [ ]:
static_fields = ["vendor", "model", "serial_nr"]
static_fields_df = df[static_fields].drop_duplicates()
assert len(static_fields_df) == 1, "There should be only a single value for static fields"
static_fields_df.iloc[0].to_dict()

In [ ]:
df.set_index("created_utc")['energy'].plot()

In [ ]:
df.charge_cycles

In [ ]:
## TODO Print out battery model

data = [
    {
        "description": "Discharge at constant power consumption. (No work done at pc)",
        "min_ts": 1706085300,
        "max_ts": 1706094734,
    },
    {
        "description": "Zugfahrt Basel - Zuerich, kein Docker Container und kein Notebook am laufen",
        "min_ts": 1707023500,
        "max_ts": 1707026694,
    },
    {
        "description": "Discharge Run 1",
        "min_ts": 1708816000,
        "max_ts": 1708819050,
    },
    {
        "description": "Discharge Run 2",
        "min_ts": 1708820000,
        "max_ts": 1708820000 + 3200,
    }
]

idx = 3

min_ts = data[idx]['min_ts']
max_ts = data[idx]['max_ts']
check_df = df.loc[(df['created_utc'] >= min_ts) & (df['created_utc'] <= max_ts)]

check_df.set_index("created_utc")['energy'].plot()
check_df.set_index("created_utc")['energy_rate'].plot()

print(f"Period from '{datetime.datetime.fromtimestamp(check_df['created_utc'].min())}' ({check_df['created_utc'].min()}) to"
      f" '{datetime.datetime.fromtimestamp(check_df['created_utc'].max())}'")

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(len(data))
dfs = []

for idx in range(len(data)):
    min_ts = data[idx]['min_ts']
    max_ts = data[idx]['max_ts']
    check_df = df.loc[(df['created_utc'] >= min_ts) & (df['created_utc'] <= max_ts)]
    dfs.append(check_df)
    check_df.set_index("created_utc")['energy'].plot(ax=axs[idx])
    check_df.set_index("created_utc")['energy_rate'].plot(ax=axs[idx])

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
model = LinearRegression()

In [ ]:
data = []
dates = {
    'min': None,
    'max': None
}
for check_df in dfs:
    if dates['min'] is None or dates['min'] > df['date'].min():
        dates['min'] = df['date'].min()
    if dates['max'] is None or dates['max'] < df['date'].max():
        dates['max'] = df['date'].max()
    y = check_df.set_index("created_utc")['energy']
    x = np.array(check_df['created_utc']).reshape((-1, 1))
    model = model.fit(x, y)
    data.append({
        "factor": model.coef_[0]/check_df.energy_rate.mean(),
        "discharge_rate": model.coef_[0],
        "power_mean": check_df.energy_rate.mean(),
        "power_std": check_df.energy_rate.std(),
    })
    

In [ ]:
battery_stats = {
    "factor_avg": pd.DataFrame(data).factor.mean(),
    "factor_std": pd.DataFrame(data).factor.std(),
    "energy_full_avg": df['energy_full'].mean(),
    "energy_full_std": df['energy_full'].std(),
}

In [ ]:
battery_stats = {
    "date": str(datetime.datetime.now().date()),
    "measuring_period": f"{dates['min'].date()} - {dates['max'].date()}",
    **static_fields_df.iloc[0].to_dict(),
    **battery_stats,
}

In [ ]:
battery_stats